# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome('/Users/andy/Desktop/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1
time.sleep(SCROLL_PAUSE_TIME)
#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")


while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    print('網頁更新中...')

print('到達頁面底端')
'''
提示：可參考以下的Stack Overflow: 
https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
'''

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


'\n提示：可參考以下的Stack Overflow: \nhttps://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166\n'

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
#print(soup.prettify())
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})#找到財經文章列表

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})#每個財經文章的最外層框
#news_block為每一個財經文章

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    #h3:
    #<a href="/news/20210305/1931684.htm" title="瀏海掰掰？蘋果光感應器新專利曝告別決心">
    #          瀏海掰掰？蘋果光感應器新專利曝告別決心
    #         </a>
    news_body = news_item.find('h3')
    #news_body.a.string為title的文字
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
     
    # 連到外部連結，擷取詳細新聞內容
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 男大生打工存80萬！網狠嗆「這只是手續費」：100萬以上才是存款

▲原PO辛苦工作存下80萬。（示意圖／記者劉維榛攝）
記者施怡妏／綜合報導
許多人辛苦賺錢，就是希望可以盡快存到人生第一桶金，一名23歲的男大生表示，存款目前有80萬但卻不快樂，雖然有時很羨慕每天吃喝玩樂的人，但很害怕過上沒錢的生活，所以很努力地賺錢，想買車買房，卻覺得這80萬根本不值得拿出來一提，沒想到底下有網友留言，「100萬以上叫存款，100萬以內叫餘額」。
一名男大生在dcard上發文，「別人都當月光族，每天出遊玩樂買一堆東西，自己卻只能羨慕他們」，但由於不想要以後過上沒有錢的日子，擔心沒辦法過自己想要的生活，所以辛苦地打工賺錢，拿到薪水就後直接把錢存到銀行裡，也慶幸自己出生在不用背負學貸的家庭，賺來的錢也都自行規劃。
▲網友表示，80萬只是他的手續費而已。（圖／翻攝自Dcard）
貼文一出後，許多網友紛紛留言鼓勵他，「錢是要拿來變成你喜歡的樣子的，不然就只是數字而已」、「23歲存80萬蠻強的啊」。也有不少網友建議他可以買股票，「錢在銀行，人在天堂，80萬ALL IN台積電吧」、「買股票，真的不會後悔，只會有相見恨晚的感覺」、「跟我存款差不多目前努力研究股票求翻身」。
‧舒緩敏感、痘痘
----------------------------------------------------------------------

[1] 蘋果春季發表會有何新品？外媒曝7大可能

▲外媒曝蘋果春季發表會有可能見到7項新品。（圖／蘋果）
記者陳俐穎／綜合報導
蘋果公司的秋季活動是業界最受期待的活動，但春季發表會也總是充滿有趣的新品。外媒BGR認為，由於疫情影響，去年的春季發表會改為線上，但所有跡象都表明今年3月會有更多新產品發布，無論有沒有實體活動，也預計將會有7款新品發布。
其中第一項就是謠傳已久的AirTags，根據以前的情報洩露，AirTag體積較小，是圓形且背面帶有金屬板。與Tile的跟踪器產品線不同，AirTags上沒有任何鑰匙孔，因此，Apple將出售帶有鑰匙扣的小型皮革袋，可以將AirTag放入其中。
其次是iPad Pro系列，根據

▲台中七期為政商中心、豪宅林立，是台中市的核心區域 。（圖／記者張菱育攝）
房產中心／綜合報導
全球各國至今仍受新冠肺炎疫情影響，而台灣因防疫成功，台商以及海外菁英大舉回流，除了為台灣各產業帶來新氣象及發展之外，大批社會高端族群、中產階級的居住需求，以及在世界經貿局勢不穩定的狀態之下，為了讓資產財富持續累積，社會中高階層的理財方向瞄準相對穩固的不動產，將錢轉換成能夠保值、甚至未來可能增值的房產，兩者皆使房市更加活絡。不論是要買房自住或置產，地段都是重要考量因素，入主精華地段還能象徵其身分地位。像台中七期就是許多台商、企業主和白領人士喜愛的區域，身處台中市政中心、坐擁百貨商圈、串連捷運交通、周邊產業園區興起，多重優勢讓七期宛如台北信義計畫區，尤其市政路上商辦、豪宅林立，成為台中七期精華中的精華。
▲七期重劃區的台中大遠百、台中新光三越商圈，假日人潮眾多。（圖／記者張菱育攝）
就在台中具指標代表性的燙金地段市政路上，出現和其他總價多在4千萬以上、中大坪數豪宅不同的物件，由興富發打造的新案「市政1號院」，僅25～41坪、2～3房，小坪數輕豪宅規劃降低總價門檻，並推出自備88萬起的輕鬆付款方案。銷售專案陳豪表示，自2021年1月正式開案以來，已售約（7成）多是海歸企業主、菁英人士為避免資產受通膨影響縮水，看準「市政1號院」精華地段各項優勢的房產價值，為自己和下一代出手投資。
▲投資不動產跟著地段、交通等方向走，是不少人推祟的黃金法則。（圖／翻攝自市政1號院官網）
交通部分，台中近來強力推動捷運，擴充城市交通網絡，捷運沿線土地和房價也因軌道經濟紛紛飆漲，「市政1號院」鄰近捷運，只需步行3分鐘即抵水安宮站，陳豪分享有台北已購客專程來買，正是依北捷經驗看好捷運有帶領區域和房價往正面發展的動能。市府機構所在地通常是城市發展中心，七期重劃區內包含台中市政府、市議會等多個公部門；生活機能方面，「市政1號院」位在市政路與文心路黃金交叉路口，車程5分鐘可至新光三越、大遠百商圈，周邊亦有家樂福、全聯旗艦店、星巴克、楓康超市等，日常生活所需完美滿足。
▲「市政1號院」距離台中捷運水安宮站、家樂福只要五分鐘路程 。
七期市政路黃金門牌給予的不只是首富繁華生活圈，市政路長1.3公里的濃蔭綠樹大道，以及府會園道、市政公園、惠順公園、惠文公園等，回家路上充滿豐沛的鮮氧綠意；明星學校惠文中學步行15分鐘

▲大立光執行長林恩平。（圖／資料照）
記者周康玉／台北報導
大立光（3008）今（5）晨公告，與先進光電營業秘密侵權案達成和解，並撤回對先進光相關案件之執行、損害賠償請求及告訴，其他部分因涉及當事人機密，就和解內容予以保密。
先進光在2011年挖角大立光工程師，利用跳槽工程師攜帶技術資料發展製程。按大立光執行長林恩平說法，此案非一般侵權案件，是先進光當初挖角公司離職員工開始竊取公司資料，整批資料盜走。
此案纏訟多年，大立光因而控告先進光侵害侵害大立光8項專利並求償140億元，2013年一審判決先進光須賠付大立光15.22億元，今年1月29日二審結果出爐、維持原判決；2月23日大立光再出招，對先進光債權提起執行假扣押，直到今日，大立光和先進光宣布和解。
大立光執行長林恩平曾表示，先進光不只透過挖角公司離職員工竊取公司資料，且一而再、再而三，甚至已經離職過去的員工還向在職員工請托，要求提供後續技術資料，一直到先進光將竊取的技術申請專利，大立光才發現。
林恩平說，先進光有意和解，還透過第三方傳話，但林恩平對於先進光處理方式不滿意，認為先進光「犯了大錯，以為道歉，就沒事」，這種態度並不可取。
----------------------------------------------------------------------

[25] Fed喊話掀股災！美股重挫345點　台積電ADR跌5.94％

記者葉國吏／綜合報導　
受到美國聯準會主席鮑爾的談話，導致美國投資人大失所望，美股道瓊重挫345點，以科技股為主的那斯達克大跌274點，台積電ADR跌5.94％、收115.59點。
週四聯準會（Fed）主席鮑爾（Jerome Powell）於《華爾街日報》網路研討會上表示，隨著經濟復甦，通膨率可能會上升，但這將是暫時的，這些壓力可能不足以刺激Fed升息。他認為目前政策立場適當，將持續目前購債步伐。
鮑爾一席話導致美股四大指數集體收黑，科技股那斯達克暴跌274點或2.11%，收在12723.47點，回吐今年漲幅；道瓊工業指數下跌345.95點或1.11%，收在30924.14點；標準普爾500指數下跌51.25點或1.34%，收在3768.47點。台積電ADR也慘跌5.94%，收115.59點。
---------------------------------------

▲金管會訂「非現金支付」交易新目標，3年後筆數成長50%，金額達到6兆。（示意圖／取自免費圖庫Pixabay）
記者陳依旻／台北報導
金管會今（4）日訂定「非現金支付」兩大新衡量目標，分別為「非現金支付交易金額」及「非現金支付交易筆數」，並將目標值設為2023年非現金支付交易筆數成長50%，交易金額達到6兆元。
金管會表示，新目標的訂定標準參考兩大方向，第一，過往幾年筆數成長約12%左右，金額約14%；第二，金管會洽詢幾家大型的發卡機構，他們預估3年後筆數成長目標為16%，金額7%，金管會銀行局副局長林志吉表示，雖然很挑戰，但會努力。
金管會於104年提出「電子化支付五年倍增計畫」，訂定109年底電子化支付比率達52%之目標，並成立推動工作小組積極推動相關措施。
據統計，109年度上開比率為40.37，但當初結構不完整，未考量到ATM轉帳也是國內民眾廣為利用於支付購物價款的管道，如果納入該交易金額，109年已接近所定109年底達52%的目標。
林志吉指出，2020年若不含ATM轉帳的話，非現金支付的交易筆數為51.64億；若包含ATM轉帳，則有52.21億筆；至於交易金額，2020年若含ATM轉帳，非現金支付的交易筆數為3.88兆，含ATM轉帳交易的話，金額為4.95兆，未來「非現金交易」都會納入ATM轉帳，要在3年後成長5成。
金管會表示，非現金支付服務具有提升經濟活動效率、節省現金支付的處理成本，也刺激民間消費帶動經濟成長、縮小地下經濟規模及提高交易透明度等優點，並補充，非現金支付難達標的原因除了未納入ATM轉帳之外，還有許多的商店雖然受疫情影響不如預期，但有些商店只接受某一種卡片，也是計畫窒礙難行的原因。
為了延續推動成效，金管會成立「提升非現金支付交易推動工作小組」，持續以「法規滾動檢討」「強化支付工具便利性」及「拓展通路運用」為三大推動主軸，積極辦理相關措施，並整合相關政府部門及金融業力量共同積極推動非現金支付交易：該工作小組於今日已召開第一次會議，經會議討論通過金管會所研擬之非現金支付的衡量指標及目標值；金管會表示，經衡酌推動非現金支付政策意義之明確性、我國支付市場環境發展的成熟度與民眾消費生活習性等因素，並參採國際衡量指標，新訂定衡量指標為「非現金支付交易金額」持續觀察我國非現金支付交易的成長情形。
--------------------------

記者吳珍儀／綜合報導
許多人有使用耳機聽音樂的習慣，但卻經常因為背景音過大，或是音樂太動感，而忽略音量調太大聲。根據蘋果聽力研究報告顯示，約有10%的人，在使用AirPods等耳機設備時，音量超過世界衛生組織WHO的安全標準，每天暴露在可能傷害聽力的音量下。
蘋果的聽力研究與密歇根大學合作透過Research進行，研究數據與世界衛生組織「Make Listening Safe」計劃共享。
蘋果於2019年11月開始進行Apple聽力研究，研究長期聲音暴露對人類所造成的影響。研究發現，約有25％的參與者每天暴露在高於世衛組織建議的音量下；50%則是在吵雜的工作場所中工作過。
這些參與者至少10年以上未接受專業聽力檢查，而在當中，約有10%被診斷出聽力受損，75%則未使用助聽器等輔助聽力的儀器，等於是「不知道」聽力已輕微受損。
世衛衛生組織預測，2050年全球將有7億多人患有某種形式的嚴重聽力損失。世界衛生組織助理總幹事任明輝表示，人們採取措施保護聽力，並適時尋求醫療協助。
----------------------------------------------------------------------

[46] 理財周刊／景氣復甦受惠股出列

文／高適
疫苗全球開打對疫情控制起了一定效果，加深市場對景氣復甦的預期，加上美國刺激方案推出在即，通膨即將來臨已成為市場新共識，拋售債券自然就是投資人的本能反射動作，債券殖利率飆升至市場擔憂的臨界值，進而引發金融資產的拋售潮，為投資人再次提供逢低進場撿便宜的好時機！
二月二五日美國十年期公債殖利率一度飆上一．六％以上，突破一．五％的市場心理關卡，並正式超越標普五百指數成分股的股息殖利率，股債殖利率反轉將降低股票的吸引力，導致資產拋售潮從債券蔓延至股票，二月二六日賣壓也從美股蔓延至亞股，外資對台股提款九四四億元，創下史上最高紀錄。
公債殖利率飆升  股市面臨壓力測試
由於美國十年期公債殖利率被許多人視為無風險利率。根據凱基投顧試算，假設殖利率為1％，股市只要每年可以帶來三％的收益，就可以抵消股市波動與潛在跌幅。但若公債殖利率升高至二％，股市收益率就必須升高至四％才有足夠的吸引力。當股市收益率調升幅度未跟上債市殖利率回升速度，自然會引發股市的賣壓。
雖說美國公債殖利率在飆高略有回檔，但在刺激方案推出的背景下，政府將大舉發

除了毋庸置疑的硬體製造及強大的供應鏈管理能力外，鴻海MIH電動車開放平台的軟實力，更是鴻海跨入電動車製造的利劍，與Fisker簽署協議，不會是最後一家，更是鴻海逆襲國際專業電動代工廠的重要一役。
文／林麗雪
去年初，ＥＭＳ製造大廠鴻海先後宣布與飛雅特克萊斯勒及國內的裕隆汽車合資，掀開了鴻海正式跨足電動車市場的扉頁，然，時值美國電動車大廠特斯拉（Tesla）聲勢正大鳴大放之際，多數人對於成立四十多年來都在做硬體代工的鴻海想要跨足汽車產業的未來，不僅不看好，且多半報以噓聲，看壞聲浪不斷。
轉型電動車的關鍵鐵三角
不過，鴻海跨入汽車製造轉型的藍圖，一張張地掀開，去年十月十六日，鴻海推出ＭＩＨ電動車開放平台，矢志成為電動車界的Android，並廣邀英雄帖，號召全球軟硬體廠加入ＭＩＨ電動車開放平台，市場這才驚覺，鴻海想跨足的汽車產業，早已顛覆傳統的造車思維。
鴻海亮出的造車利劍，不僅僅止於推出軟硬兼施的電動車開放平台。今年一月鴻海宣布，在汽車產業已有四十年經驗的鄭顯聰，擔任ＭＩＨ聯盟執行長；鄭顯聰除了曾任職福特、飛雅特外，亦是蔚來汽車（NIO）的聯合創辦人，他加入鴻海，除了專責ＭＩＨ聯盟規劃外，並將掌管硬體部門及平台的規格定義，其在汽車產業經驗跨足傳統與未來，補足了市場對鴻海硬體製造經驗恐不足的疑慮。
ＭＩＨ聯盟的另一關鍵人物，則是魏國章，他過去是賈伯斯創辦的NeXT Computer中唯一的華人核心軟體成員，在蘋果收購NeXT之後，魏國章跟隨賈伯斯團隊進入蘋果，開發出iOS和Mac OS X作業系統，未來魏國章將主導ＭＩＨ的軟體平台。
鄭顯聰、魏國章兩位軟硬體大將到位，和以電子工程背景出身的鴻海董事長劉揚偉，形成鴻海推進電動車事業轉型的關鍵鐵三角，有了對的人、對的團隊及對的營運模式，鴻海的電動車轉型大計，在不到一年的時間，已有先馳得點的態勢。
鴻海日前宣布拿下第六家電動車廠盟約，和美國電動車新創公司Fisker將透過「Project PEAR」專案，於二○二三年第四季正式量產一台電動車。Fisker的第一台電動車款Ocean SUV本來擬在福斯的電動車ＭＥＢ平台打造，據了解，最後由於量產不順，改由專業電動車代工廠Magna取得，Magna手上客戶還有BMW、Toyota、Jaguar等，號稱已量產超過三七○萬輛電動車，這次鴻海拿下Fisker第二款電動車款代工，雙方預計在第

▲外資今日賣超525.15億元，再度刷新賣超前五名紀錄。（圖／記者屠惠剛攝）
記者林妤柔／綜合報導
美國公債殖利率再度攀升，美股全面下跌，拖累台股今（4）日重挫逾300點，失守萬六大關。其中，三大法人合計賣超551.9億元，外資賣超525.15億元，再度刷新外資賣超前5名紀錄，擠掉2007年8月1日，美國次貸危機賣超489億的紀錄。
外資賣超前5名紀錄
----------------------------------------------------------------------

[67] 永豐餘投控2020年股利出爐　每張配發1500元

▲永豐餘今召開法說會。（圖／記者余弦妙攝）
記者余弦妙／台北報導
永豐餘投控(1907)今(4)日召開法說會，說明2020年營運概況，永豐餘2020年度合併營業收入為新台幣732.34億，全年合併營業淨利56.03億，年增87.3%。2020年每股稅後盈餘新台幣3.14元，較2019年成長127.5%。董事會決議每股配發現金股利新台幣1.50元，也較前一年度成長114.2%。
永豐餘表示，2020年工紙紙器事業群因台灣、越南供應鏈移轉效應，出貨成長；揚州工紙因應下半年市場需求提升，調整產品策略，第四季營運翻轉。疫情下，消費者對各類生活用紙品、潔品及醫療乳膠手套需求提升，永豐餘消費品實業、申豐特用應材業績皆受惠提升。
再加上，因為疫情關係外送服務需求增加，造成一次性食品容器使用量大增，中華紙漿所推的益利CircuWell系列非塑可全回收循環紙品，期能解決一般含塑食品容器難以回收處理問題。
認列於營業外收支的永豐金及其他投資股利收入增加，加上元太科技等權益法投資收益提升，以及於本年度完成江陰土地使用權出售及永豐餘消費品實業出售北京廠利益，挹注稅後淨利。永豐餘投控2020年底每股淨值為27.09元，成長14.6%。
展望2021年，永豐餘說，漿紙事業群面對漿價波動的成本壓力，將積極發展纖維材料應用，桃園觀音新廠建構為研發生產基地及倉儲轉運中心，持續拓展益利非塑應用及提高對客戶的服務優勢。
而在工紙紙器事業群營運上繼續拓展越南營運，優化台灣、越南、中國三地產品結構；擴大投資台灣沼氣發電及SRF(Solid Recovered Fuel)鍋爐產汽發電，落實全循環經濟理念， 「不僅循環，更增獲利。」消費品事業群持續優化客戶及產

▲宏泰人壽今（4）日針對熱銷醫療險傳出想調漲，卻在業界掀起「保單之亂」一事，強調外界批評是「張冠李戴」。（圖／宏泰人壽提供）
記者陳依旻／台北報導
宏泰人壽今（4）日針對熱銷醫療險想調漲，卻在業界掀起「保單之亂」一事強調外界批評是「張冠李戴」；第一，該險種續期保費沒有漲價；第二，目前已在3月1日就停售。
宏泰人壽在2019年推出薰衣草醫療險（商品代號HAS），由於價格平易近人，有保戶稱它為「最強醫療實支險」，業者指出，近2年在市場上相當熱賣，因為住院可同時領取住院日額及病房費限額理賠的「宏泰人壽薰衣草醫療健康保險附約」。
但宏泰人壽2月中發公函給銷售通路，內容表示保險成本上升，這張保單要在3月22日變更費率計算續期保費，不少保戶發現費率將狂漲超過100%，紛紛發出不平之鳴。
有保戶指出，保費調漲的問題，舊保戶保費調漲在法理上本來就可以，但是宏泰想要3月起，調漲超過100%，「宏泰說是因應保費調整機制要調漲，但是幅度大到一個誇張，以前有過也都是微調10到20%而已」；業界分析，可能是想整頓財務，因為保發中心所公布的壽險業半年報，宏泰人壽淨值低於3%，金管會也曾表態，不建議宏泰人壽以不動產作價增資。
《ETtoday新聞雲》記者當時就宏泰人壽此商品保費一次調漲逾100%有無異常一事，訪問金管會保險局官員，官員表示，已向宏泰人壽了解，在銷售通路端做招攬時，有無說明費率並非保證不動，但宏泰表示「續保件」不會漲價；相關質疑尚未平息，宏泰人壽原打算在3月中旬漲調續期保費，結果在3月3日傍晚於官網發布最新聲明，指出這張保單於3月1日停售且不調漲保費，宏泰人壽指出，外界很多聲音都是對公文函及附件張冠李戴，站在公司的立場認為毋須澄清，不再多說。
----------------------------------------------------------------------

[81] WhatsApp新功能曝光！相片影片「閱後即焚」

▲Whatapp將開發閱後即焚功能。（圖／取自MacRumors）
記者陳俐穎／綜合報導
根據外媒MacRumors報導，通訊軟體WhatsApp正在測試一項新功能，用戶可以設定「閱後即焚」模式，收到圖像的對方只能查看一次，也無法儲存照片。
國外網站 WABetainfo發現此新功能，發送者可以在更新後的界面中選擇要閱後即焚的圖像，如果選擇

記者蔡紹堅／綜合報導
中芯國際（SMIC）3日晚間在港交所公告，已與荷蘭光刻機廠艾司摩爾（ASML）簽訂金額達12億美元的購買單，所購買的項目為「用於生產晶圓的機器」，很高機率就是光刻機。
公告提到，據批量採購協議已於2020年3月16日至2021年3月2日的12個月期間，就購買艾司摩爾產品與艾司摩爾集團簽訂購買單。
公告顯示，上述阿艾司摩爾購買單為該期間艾司摩爾集團向中芯國際供應「用於生產晶圓的機器」而作出，艾司摩爾購買單的總代價為12.02億美元。
由於艾司摩爾的採購協議先前已到期，雙方也將原先的採購協議從2018年1月1日至2020年12月31日延長至從2018年1月1日至2021年12月31日。
協議中並未明確指出12億美元的產品為何，但有提到為「用於生產晶圓的機器」，艾司摩爾為光刻機廠龍頭，因此有很高的機率為艾司摩爾所生產的光刻機。
陸媒分析，艾司摩爾的EUV（極紫外光刻技術），能推進中芯國際7nm/5nm製程，而其DUV（深紫外線光科技術）則支撐著中芯國際傳統產線的建造和擴產，例如公司在今年最被市場看好的8吋晶片產能，就來自傳統產線。
----------------------------------------------------------------------

[94] 失守萬六！台股暴跌逾300點　聯發科跌破900元

▲台股失守萬六關卡。（圖／記者屠惠剛攝）
記者林妤柔／綜合報導
台股今（4）日以16091點開出，早盤跌逾160點，截至10點38分，指數跌幅擴大，跌306點或1.89%，指數暫收15905點，失守萬六關卡。
權值股部分，台積電（2330）跌2.89%，陷入600元保衛戰；聯發科（2454）在900元附近震盪；聯電（2303）跌3.76%，在50元上下震盪。
大立光（3008）跌1.6%、鴻海（2317）跌1.73%。
----------------------------------------------------------------------

[95] 年收3000萬！HTC天才設計師吃裡扒外　宏達電內鬼案輕判緩刑

▲HTC前首席設計師簡志霖，二審獲判緩刑。（圖／CFP）
記者吳銘峯／台北報導
HTC宏達電2012年間爆發內鬼案，時任工業設計部副總經理兼首席設計師簡志霖，任內私自重製公司的商業機密Ic

In [6]:
browser.close()